In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
import torch 
import torch.nn as nn 
from utils_preprocess import make_dataframe
from models import NetworkStructure
from utils_train import ModelTrainer, plot_loss_history, calc_r2

%matplotlib inline 

C:\Users\zhxch\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
'''Set numpy random seed'''

numpy_seed = 999
np.random.seed(numpy_seed)


'''Prepare channel flow data'''

Retau_train = [1000, 2000, 5200] 
Retau_test = [550]

df_train = pd.concat([make_dataframe(Retau) for Retau in Retau_train])
df_test = pd.concat([make_dataframe(Retau) for Retau in Retau_test])

df_train['a'] = df_train[['a_uv', 'a_uu', 'a_vv', 'a_ww']].values.tolist()
df_test['a'] = df_test[['a_uv', 'a_uu', 'a_vv', 'a_ww']].values.tolist()

# scale up target variable in order for stable training, will consider implementing automatic scaler 
df_train['a_uv'] *= 1e3   
df_test['a_uv'] *= 1e3 


'''Split training data into training and validation data'''

val_fraction = 0.2
df_train, df_val = train_test_split(df_train, test_size=val_fraction, shuffle=True) 

# # use following approach to imitate pytorch random_split function (which was used in earlier experiments)
# from torch import randperm 
# torch.manual_seed(999)
# indices = randperm(len(df_train)).tolist()
# train_indices = indices[:int((1-val_fraction)*len(df_train))]
# val_indices = indices[int((1-val_fraction)*len(df_train)):]
# df_val = df_train.iloc[val_indices]
# df_train = df_train.iloc[train_indices]


'''Print number of data points'''

print('N_train: {}\nN_val: {}\nN_test: {}'.format(len(df_train), len(df_val), len(df_test)))

N_train: 1126
N_val: 282
N_test: 192


In [3]:
df_train.head(5)

,y+,y,index,du_dy,a_uv,a_uu,a_vv,a_ww,Re_tau,DU_DY,Y,a
56,54.730663,0.010554,56,9.318907,-1.626941,0.004926,-0.003841,-0.001085,5185.897,"[215.14682814005528, 215.1435270334813, 215.13...","[0.0, 1.3710713922450184e-05, 4.16995405351281...","[-0.0016269412300123857, 0.004926263300954793,..."
576,3252.192145,0.627122,576,0.196459,-0.636307,0.001040,-0.000649,-0.000392,5185.897,"[215.14682814005528, 215.1435270334813, 215.13...","[0.0, 1.3710713922450184e-05, 4.16995405351281...","[-0.0006363065051346082, 0.001040375549731483,..."
744,4942.830131,0.953129,744,0.029074,-0.079696,0.000356,-0.000185,-0.000171,5185.897,"[215.14682814005528, 215.1435270334813, 215.13...","[0.0, 1.3710713922450184e-05, 4.16995405351281...","[-7.969594778035911e-05, 0.0003559033069961584..."
500,2547.326692,0.491203,500,0.251217,-0.869860,0.001453,-0.000926,-0.000527,5185.897,"[215.14682814005528, 215.1435270334813, 215.13...","[0.0, 1.3710713922450184e-05, 4.16995405351281...","[-0.000869859846102128, 0.0014527322323839662,..."
158,319.879688,0.061683,158,1.741034,-1.599301,0.004056,-0.002799,-0.001257,5185.897,"[215.14682814005528, 215.1435270334813, 215.13...","[0.0, 1.3710713922450184e-05, 4.16995405351281...","[-0.001599301010904181, 0.004055674757247312, ..."


In [4]:
torch_seed = 999
torch.manual_seed(torch_seed)

def evaluate(trainer, df):
    preds, targets = trainer.predict(df)
    return {r'$R^2$': calc_r2(targets, preds)}

'''Set target variable'''
target_label = ['a_uv'] # or ['a'] (output_len = 1 if 'a_uv'; output_len = 4 if 'a')


'''Define model'''
# model_name: ['MLP', 'MLP-Prior', 'MLP-BC', 'MLP-Re', 'MLP-NL', 'MLP-Re-Prior', 'MLP-BC-Re', 'MLP-BC-Re-Prior', 'MLP-BC-Re-NL']
# model_kwargs is a dictionary of model hyperparameters  
# Re models need 'Re_input_layers', a tuple of layer indices that specifies at which layers to input Re_tau
# NL models need 'alphanet_structure' (structure of the network predicting alpha from y+) 
# and 'alpha0' (boundary condition for alpha at y+ = 0, can only be 0 or 1, default is 1) 
# Prior models need 'scale'

model_MLP = 'MLP'
model_MLP_kwargs = {'structure': 
                    NetworkStructure().set_num_layers(5).set_num_nodes(50).set_output_len(1).set_activation(nn.ELU())}

model_MLP_Re = 'MLP-Re'
model_MLP_Re_kwargs = {'structure': 
                       NetworkStructure().set_num_layers(5).set_num_nodes(50).set_output_len(1).set_activation(nn.ELU()),
                       'Re_input_layers': (3,)}

model_MLP_BC_Re = 'MLP-BC-Re'
model_MLP_BC_Re_kwargs = {'structure': 
                       NetworkStructure().set_num_layers(5).set_num_nodes(50).set_output_len(1).set_activation(nn.ELU()),
                       'Re_input_layers': (3,)}

model_MLP_BC_Re_NL = 'MLP-BC-Re-NL'
model_MLP_BC_Re_NL_kwargs = {'structure': 
                       NetworkStructure().set_num_layers(5).set_num_nodes(50).set_output_len(1).set_activation(nn.ELU()),
                       'Re_input_layers': (3,),
                       'alphanet_structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_activation(nn.ELU()),
                       'alpha0': 1}

model1 = 'MLP-BC-Re-Prior'
model1_kwargs = {'structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_output_len(1).set_activation(nn.ELU()),
     'Re_input_layers': (3,),
#     'alphanet_structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_activation(nn.ELU()),
#     'alpha0': 1,
     'scale': 1,  
               }

model2 = 'MLP-BC-Re-Prior'
model2_kwargs = {'structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_output_len(1).set_activation(nn.ELU()),
     'Re_input_layers': (3,),
#     'alphanet_structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_activation(nn.ELU()),
#     'alpha0': 1,
     'scale': 2,  
               }

model3 = 'MLP-BC-Re-Prior'
model3_kwargs = {'structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_output_len(1).set_activation(nn.ELU()),
     'Re_input_layers': (3,),
#     'alphanet_structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_activation(nn.ELU()),
#     'alpha0': 1,
     'scale': 3,  
               }

model5 = 'MLP-BC-Re-Prior'
model5_kwargs = {'structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_output_len(1).set_activation(nn.ELU()),
     'Re_input_layers': (3,),
#     'alphanet_structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_activation(nn.ELU()),
#     'alpha0': 1,
     'scale': 5,  
               }

model7 = 'MLP-BC-Re-Prior'
model7_kwargs = {'structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_output_len(1).set_activation(nn.ELU()),
     'Re_input_layers': (3,),
#     'alphanet_structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_activation(nn.ELU()),
#     'alpha0': 1,
     'scale': 7,  
               }

model10 = 'MLP-BC-Re-Prior'
model10_kwargs = {'structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_output_len(1).set_activation(nn.ELU()),
     'Re_input_layers': (3,),
#     'alphanet_structure': NetworkStructure().set_num_layers(5).set_num_nodes(50).set_activation(nn.ELU()),
#     'alpha0': 1,
     'scale': 10}

'''Define test case'''
df=df_test

In [ ]:
####################### MLP #########################
print('MLP')
MLP_loss=[]
MLP_preds=[]

MLP = ModelTrainer()
MLP.set_target_variable(target_label)
MLP.set_model(model_MLP, **model_MLP_kwargs)
MLP.set_criterion(nn.MSELoss())
MLP.set_optimizer('Adam', lr=1e-6)
MLP.set_config(batch_size=10, print_freq=200, max_epochs=10000, earlystopping=True, patience=30)
   
MLP_loss.append(MLP.fit(df_train, df_val, df_test))
    
preds, targets = MLP.predict(df)
MLP_preds.append(preds)
    
scores = {'Train': evaluate(MLP, df_train),
          'Val': evaluate(MLP, df_val),
          'Test': evaluate(MLP, df_test)}
print('Loss:')
print(pd.DataFrame(scores).round(2))
print('----------------------------------------------------------------')

MLP_preds = np.array(MLP_preds)
MLP_mean = MLP_preds.mean(axis=0)
MLP_std = MLP_preds.std(axis=0)
MLP_mean = MLP_mean.reshape(len(MLP_mean),)
MLP_std = MLP_std.reshape(len(MLP_std),)

####################### MLP-Re #########################
print('MLP-Re')
MLP_Re_loss=[]
MLP_Re_preds=[]

MLP_Re = ModelTrainer()
MLP_Re.set_target_variable(target_label)
MLP_Re.set_model(model_MLP_Re, **model_MLP_Re_kwargs)
MLP_Re.set_criterion(nn.MSELoss())
MLP_Re.set_optimizer('Adam', lr=1e-6)
MLP_Re.set_config(batch_size=10, print_freq=200, max_epochs=10000, earlystopping=True, patience=30)
   
MLP_Re_loss.append(MLP_Re.fit(df_train, df_val, df_test))
    
preds, targets = MLP_Re.predict(df)
MLP_Re_preds.append(preds)
    
scores = {'Train': evaluate(MLP_Re, df_train),
          'Val': evaluate(MLP_Re, df_val),
          'Test': evaluate(MLP_Re, df_test)}
print('Loss:')
print(pd.DataFrame(scores).round(2))
print('----------------------------------------------------------------')

MLP_Re_preds = np.array(MLP_Re_preds)
MLP_Re_mean = MLP_Re_preds.mean(axis=0)
MLP_Re_std = MLP_Re_preds.std(axis=0)
MLP_Re_mean = MLP_Re_mean.reshape(len(MLP_Re_mean),)
MLP_Re_std = MLP_Re_std.reshape(len(MLP_Re_std),)

####################### MLP-BC-Re #########################
print('MLP-BC-Re')
MLP_BC_Re_loss=[]
MLP_BC_Re_preds=[]

MLP_BC_Re = ModelTrainer()
MLP_BC_Re.set_target_variable(target_label)
MLP_BC_Re.set_model(model_MLP_BC_Re, **model_MLP_BC_Re_kwargs)
MLP_BC_Re.set_criterion(nn.MSELoss())
MLP_BC_Re.set_optimizer('Adam', lr=1e-6)
MLP_BC_Re.set_config(batch_size=10, print_freq=200, max_epochs=10000, earlystopping=True, patience=30)
   
MLP_BC_Re_loss.append(MLP_BC_Re.fit(df_train, df_val, df_test))
    
preds, targets = MLP_BC_Re.predict(df)
MLP_BC_Re_preds.append(preds)
    
scores = {'Train': evaluate(MLP_BC_Re, df_train),
          'Val': evaluate(MLP_BC_Re, df_val),
          'Test': evaluate(MLP_BC_Re, df_test)}
print('Loss:')
print(pd.DataFrame(scores).round(2))
print('----------------------------------------------------------------')

MLP_BC_Re_preds = np.array(MLP_BC_Re_preds)
MLP_BC_Re_mean = MLP_BC_Re_preds.mean(axis=0)
MLP_BC_Re_std = MLP_BC_Re_preds.std(axis=0)
MLP_BC_Re_mean = MLP_BC_Re_mean.reshape(len(MLP_BC_Re_mean),)
MLP_BC_Re_std = MLP_BC_Re_std.reshape(len(MLP_BC_Re_std),)

####################### MLP-BC-Re-NL #########################
#print('MLP-BC-Re-NL')
#MLP_BC_Re_NL_loss=[]
#MLP_BC_Re_NL_preds=[]

#MLP_BC_Re_NL = ModelTrainer()
#MLP_BC_Re_NL.set_target_variable(target_label)
#MLP_BC_Re_NL.set_model(model_MLP_BC_Re_NL, **model_MLP_BC_Re_NL_kwargs)
#MLP_BC_Re_NL.set_criterion(nn.MSELoss())
#MLP_BC_Re_NL.set_optimizer('Adam', lr=1e-6)
#MLP_BC_Re_NL.set_config(batch_size=10, print_freq=200, max_epochs=1000, earlystopping=True, patience=30)
   
#MLP_BC_Re_NL_loss.append(MLP_BC_Re_NL.fit(df_train, df_val, df_test))
    
#preds, targets = MLP_BC_Re_NL.predict(df)
#MLP_BC_Re_NL_preds.append(preds)
    
#scores = {'Train': evaluate(MLP_BC_Re_NL, df_train),
#          'Val': evaluate(MLP_BC_Re_NL, df_val),
#          'Test': evaluate(MLP_BC_Re_NL, df_test)}
#print('Loss:')
#print(pd.DataFrame(scores).round(2))
#print('----------------------------------------------------------------')

#MLP_BC_Re_NL_preds = np.array(MLP_BC_Re_NL_preds)
#MLP_BC_Re_NL_mean = MLP_BC_Re_NL_preds.mean(axis=0)
#MLP_BC_Re_NL_std = MLP_BC_Re_NL_preds.std(axis=0)
#MLP_BC_Re_NL_mean = MLP_BC_Re_NL_mean.reshape(len(MLP_BC_Re_NL_mean),)
#MLP_BC_Re_NL_std = MLP_BC_Re_NL_std.reshape(len(MLP_BC_Re_NL_std),)

####################### Bootstrap MLP-BC-Re-Prior=1 #########################
print('Bootstrap MLP-BC-Re Prior=1')
BootMLP_Prior1_loss=[]
BootMLP_Prior1_preds=[]
BootMLP_Prior1_targets=[]

ensemble = 5
for i in range(ensemble):
    print('ensemble {}'.format(i+1))
    BootMLP_Prior1 = ModelTrainer()
    BootMLP_Prior1.set_target_variable(target_label)
    BootMLP_Prior1.set_model(model1, **model1_kwargs)
    BootMLP_Prior1.set_criterion(nn.MSELoss())
    BootMLP_Prior1.set_optimizer('Adam', lr=1e-6)
    BootMLP_Prior1.set_config(batch_size=10, print_freq=200, max_epochs=1000, earlystopping=True, patience=30)
    
    BootMLP_Prior1_loss.append(BootMLP_Prior1.fit(df_train, df_val, df_test))
    
    preds, targets = BootMLP_Prior1.predict(df)
    BootMLP_Prior1_preds.append(preds)
    BootMLP_Prior1_targets.append(targets)
    
    scores = {'Train': evaluate(BootMLP_Prior1, df_train),
              'Val': evaluate(BootMLP_Prior1, df_val),
              'Test': evaluate(BootMLP_Prior1, df_test)}
    print('Loss:')
    print(pd.DataFrame(scores).round(2))
    print('----------------------------------------------------------------')

BootMLP_Prior1_preds = np.array(BootMLP_Prior1_preds)
BootMLP_Prior1_mean = BootMLP_Prior1_preds.mean(axis=0)
BootMLP_Prior1_std = BootMLP_Prior1_preds.std(axis=0)
BootMLP_Prior1_mean = BootMLP_Prior1_mean.reshape(len(BootMLP_Prior1_mean),)
BootMLP_Prior1_std = BootMLP_Prior1_std.reshape(len(BootMLP_Prior1_std),)

####################### Bootstrap MLP-BC-Re-Prior=2 #########################
print('Bootstrap MLP-BC-Re Prior=2')
BootMLP_Prior2_loss=[]
BootMLP_Prior2_preds=[]
BootMLP_Prior2_targets=[]

ensemble = 5
for i in range(ensemble):
    print('ensemble {}'.format(i+1))
    BootMLP_Prior2 = ModelTrainer()
    BootMLP_Prior2.set_target_variable(target_label)
    BootMLP_Prior2.set_model(model2, **model2_kwargs)
    BootMLP_Prior2.set_criterion(nn.MSELoss())
    BootMLP_Prior2.set_optimizer('Adam', lr=1e-6)
    BootMLP_Prior2.set_config(batch_size=10, print_freq=200, max_epochs=1000, earlystopping=True, patience=30)
    
    BootMLP_Prior2_loss.append(BootMLP_Prior2.fit(df_train, df_val, df_test))
    
    preds, targets = BootMLP_Prior2.predict(df)
    BootMLP_Prior2_preds.append(preds)
    BootMLP_Prior2_targets.append(targets)
    
    scores = {'Train': evaluate(BootMLP_Prior2, df_train),
              'Val': evaluate(BootMLP_Prior2, df_val),
              'Test': evaluate(BootMLP_Prior2, df_test)}
    print('Loss:')
    print(pd.DataFrame(scores).round(2))
    print('----------------------------------------------------------------')

BootMLP_Prior2_preds = np.array(BootMLP_Prior2_preds)
BootMLP_Prior2_mean = BootMLP_Prior2_preds.mean(axis=0)
BootMLP_Prior2_std = BootMLP_Prior2_preds.std(axis=0)
BootMLP_Prior2_mean = BootMLP_Prior2_mean.reshape(len(BootMLP_Prior2_mean),)
BootMLP_Prior2_std = BootMLP_Prior2_std.reshape(len(BootMLP_Prior2_std),)

####################### Bootstrap MLP-BC-Re-Prior=3 #########################
print('Bootstrap MLP-BC-Re Prior=3')
BootMLP_Prior3_loss=[]
BootMLP_Prior3_preds=[]
BootMLP_Prior3_targets=[]

ensemble = 5
for i in range(ensemble):
    print('ensemble {}'.format(i+1))
    BootMLP_Prior3 = ModelTrainer()
    BootMLP_Prior3.set_target_variable(target_label)
    BootMLP_Prior3.set_model(model3, **model3_kwargs)
    BootMLP_Prior3.set_criterion(nn.MSELoss())
    BootMLP_Prior3.set_optimizer('Adam', lr=1e-6)
    BootMLP_Prior3.set_config(batch_size=10, print_freq=200, max_epochs=10000, earlystopping=True, patience=30)
    
    BootMLP_Prior3_loss.append(BootMLP_Prior3.fit(df_train, df_val, df_test))
    
    preds, targets = BootMLP_Prior3.predict(df)
    BootMLP_Prior3_preds.append(preds)
    BootMLP_Prior3_targets.append(targets)
    
    scores = {'Train': evaluate(BootMLP_Prior3, df_train),
              'Val': evaluate(BootMLP_Prior3, df_val),
              'Test': evaluate(BootMLP_Prior3, df_test)}
    print('Loss:')
    print(pd.DataFrame(scores).round(2))
    print('----------------------------------------------------------------')

BootMLP_Prior3_preds = np.array(BootMLP_Prior3_preds)
BootMLP_Prior3_mean = BootMLP_Prior3_preds.mean(axis=0)
BootMLP_Prior3_std = BootMLP_Prior3_preds.std(axis=0)
BootMLP_Prior3_mean = BootMLP_Prior3_mean.reshape(len(BootMLP_Prior3_mean),)
BootMLP_Prior3_std = BootMLP_Prior3_std.reshape(len(BootMLP_Prior3_std),)

####################### Bootstrap MLP-BC-Re-Prior=5 #########################
print('Bootstrap MLP-BC-Re Prior=5')
BootMLP_Prior5_loss=[]
BootMLP_Prior5_preds=[]
BootMLP_Prior5_targets=[]

ensemble = 5
for i in range(ensemble):
    print('ensemble {}'.format(i+1))
    BootMLP_Prior5 = ModelTrainer()
    BootMLP_Prior5.set_target_variable(target_label)
    BootMLP_Prior5.set_model(model5, **model5_kwargs)
    BootMLP_Prior5.set_criterion(nn.MSELoss())
    BootMLP_Prior5.set_optimizer('Adam', lr=1e-6)
    BootMLP_Prior5.set_config(batch_size=10, print_freq=200, max_epochs=1000, earlystopping=True, patience=30)
    
    BootMLP_Prior5_loss.append(BootMLP_Prior5.fit(df_train, df_val, df_test))
    
    preds, targets = BootMLP_Prior5.predict(df)
    BootMLP_Prior5_preds.append(preds)
    BootMLP_Prior5_targets.append(targets)
    
    scores = {'Train': evaluate(BootMLP_Prior5, df_train),
              'Val': evaluate(BootMLP_Prior5, df_val),
              'Test': evaluate(BootMLP_Prior5, df_test)}
    print('Loss:')
    print(pd.DataFrame(scores).round(2))
    print('----------------------------------------------------------------')

BootMLP_Prior5_preds = np.array(BootMLP_Prior5_preds)
BootMLP_Prior5_mean = BootMLP_Prior5_preds.mean(axis=0)
BootMLP_Prior5_std = BootMLP_Prior5_preds.std(axis=0)
BootMLP_Prior5_mean = BootMLP_Prior5_mean.reshape(len(BootMLP_Prior5_mean),)
BootMLP_Prior5_std = BootMLP_Prior5_std.reshape(len(BootMLP_Prior5_std),)

####################### Bootstrap MLP-BC-Re-Prior=7 #########################
print('Bootstrap MLP-BC-Re Prior=7')
BootMLP_Prior7_loss=[]
BootMLP_Prior7_preds=[]
BootMLP_Prior7_targets=[]

ensemble = 5
for i in range(ensemble):
    print('ensemble {}'.format(i+1))
    BootMLP_Prior7 = ModelTrainer()
    BootMLP_Prior7.set_target_variable(target_label)
    BootMLP_Prior7.set_model(model7, **model7_kwargs)
    BootMLP_Prior7.set_criterion(nn.MSELoss())
    BootMLP_Prior7.set_optimizer('Adam', lr=1e-6)
    BootMLP_Prior7.set_config(batch_size=10, print_freq=200, max_epochs=1000, earlystopping=True, patience=30)
    
    BootMLP_Prior7_loss.append(BootMLP_Prior7.fit(df_train, df_val, df_test))
    
    preds, targets = BootMLP_Prior7.predict(df)
    BootMLP_Prior7_preds.append(preds)
    BootMLP_Prior7_targets.append(targets)
    
    scores = {'Train': evaluate(BootMLP_Prior7, df_train),
              'Val': evaluate(BootMLP_Prior7, df_val),
              'Test': evaluate(BootMLP_Prior7, df_test)}
    print('Loss:')
    print(pd.DataFrame(scores).round(2))
    print('----------------------------------------------------------------')

BootMLP_Prior7_preds = np.array(BootMLP_Prior7_preds)
BootMLP_Prior7_mean = BootMLP_Prior7_preds.mean(axis=0)
BootMLP_Prior7_std = BootMLP_Prior7_preds.std(axis=0)
BootMLP_Prior7_mean = BootMLP_Prior7_mean.reshape(len(BootMLP_Prior7_mean),)
BootMLP_Prior7_std = BootMLP_Prior7_std.reshape(len(BootMLP_Prior7_std),)

####################### Bootstrap MLP-BC-Re-Prior=10 #########################
print('Bootstrap MLP-BC-Re Prior=10')
BootMLP_Prior10_loss=[]
BootMLP_Prior10_preds=[]
BootMLP_Prior10_targets=[]

ensemble = 5
for i in range(ensemble):
    print('ensemble {}'.format(i+1))
    BootMLP_Prior10 = ModelTrainer()
    BootMLP_Prior10.set_target_variable(target_label)
    BootMLP_Prior10.set_model(model10, **model10_kwargs)
    BootMLP_Prior10.set_criterion(nn.MSELoss())
    BootMLP_Prior10.set_optimizer('Adam', lr=1e-6)
    BootMLP_Prior10.set_config(batch_size=10, print_freq=200, max_epochs=1000, earlystopping=True, patience=30)
    
    BootMLP_Prior10_loss.append(BootMLP_Prior10.fit(df_train, df_val, df_test))
    
    preds, targets = BootMLP_Prior10.predict(df)
    BootMLP_Prior10_preds.append(preds)
    BootMLP_Prior10_targets.append(targets)
    
    scores = {'Train': evaluate(BootMLP_Prior10, df_train),
              'Val': evaluate(BootMLP_Prior10, df_val),
              'Test': evaluate(BootMLP_Prior10, df_test)}
    print('Loss:')
    print(pd.DataFrame(scores).round(2))
    print('----------------------------------------------------------------')

BootMLP_Prior10_preds = np.array(BootMLP_Prior10_preds)
BootMLP_Prior10_mean = BootMLP_Prior10_preds.mean(axis=0)
BootMLP_Prior10_std = BootMLP_Prior10_preds.std(axis=0)
BootMLP_Prior10_mean = BootMLP_Prior10_mean.reshape(len(BootMLP_Prior10_mean),)
BootMLP_Prior10_std = BootMLP_Prior10_std.reshape(len(BootMLP_Prior10_std),)

MLP

 epoch    time(s)      train_loss        val_loss       test_loss
     0       1.12         1.72342         1.59737         2.79137
   200       0.18         0.18482         0.16389         0.65650
   400       0.24         0.09830         0.09429         0.41235
   600       0.20         0.06128         0.05557         0.32667
   800       0.17         0.04232         0.03529         0.29910
  1000       0.17         0.03757         0.03087         0.30539
  1200       0.17         0.03624         0.02987         0.31097
  1400       0.17         0.03555         0.02937         0.31312
  1600       0.23         0.03514         0.02904         0.31473
  1800       0.17         0.03484         0.02885         0.31484
  2000       0.17         0.03460         0.02872         0.31459
  2159       0.18         0.03449         0.02869         0.31468

Terminated training for early stopping at Epoch 2159

Total training time: 409.96 sec
Loss:
       Train   Val  Test
$R^2$   0.91  0.91 

  1000       0.48         0.01316         0.01153         0.13241

Total training time: 614.19 sec
Loss:
       Train   Val  Test
$R^2$   0.97  0.97  0.83
----------------------------------------------------------------
ensemble 2

 epoch    time(s)      train_loss        val_loss       test_loss
     0       0.49       145.56868       123.43536        11.38736
   200       0.50         0.07583         0.07674         0.24801
   400       0.81         0.03905         0.03730         0.15882
   600       0.48         0.02035         0.01795         0.13378
   800       0.55         0.01264         0.01144         0.12438
  1000       0.49         0.00979         0.00847         0.11575

Total training time: 583.21 sec
Loss:
       Train   Val  Test
$R^2$   0.97  0.97  0.85
----------------------------------------------------------------
ensemble 3

 epoch    time(s)      train_loss        val_loss       test_loss
     0       0.58     13703.30571     14020.67814       184.78816
   200  

In [ ]:
fig = plt.figure(figsize=[12,10], dpi=300)
fig.suptitle('Uncertainty Estimates by Bootstrapped Ensemble', verticalalignment='center')
###############################################################################
plt.subplot(2, 2, 1)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Predicted Mean with 90% Uncertainty Bound', fontsize=12)
plt.xlim(-100, 800)
plt.ylim(-4, 0.5)
# predictive mean and deviation
plt.plot(df['y+'], MLP_mean, 'k-', linewidth=1.5, label='MLP Prediction')
plt.plot(df['y+'], MLP_Re_mean, 'g:', linewidth=1.5, label='MLP-Re Prediction')
plt.plot(df['y+'], MLP_BC_Re_mean, 'c:', linewidth=1.5, label='MLP-BC-Re Prediction')
#plt.plot(df['y+'], MLP_BC_Re_NL_mean, 'm:', linewidth=1.5, label='MLP-BC-Re-NL Prediction')
plt.plot(df['y+'], BootMLP_Prior1_mean, 'b--', linewidth=1.5, label='MLP-BC-Re-Prior1 Prediction')
plt.fill_between(df['y+'], BootMLP_Prior1_mean - BootMLP_Prior1_std, BootMLP_Prior1_mean + BootMLP_Prior1_std, alpha=0.5, color='red')
plt.fill_between(df['y+'], BootMLP_Prior1_mean + 1.64*BootMLP_Prior1_std, BootMLP_Prior1_mean - 1.64*BootMLP_Prior1_std, alpha=0.15, color='red') # 90% uncertainty: 1.64 std
plt.legend(loc='lower left')
plt.xlabel('y+'); plt.ylabel('$a_{uv}$')
###############################################################################
plt.subplot(2, 2, 2)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Prediction of Each Ensemble', fontsize=12)
plt.xlabel('y+'); plt.ylabel('$a_{uv}$')
plt.legend(loc='lower left')

# loop for predictions of each head 
for i in range(ensemble):
    plt.plot(df['y+'], BootMLP_Prior1_preds[i,:], linestyle='--', linewidth=1.5)
plt.xlim(-100, 800)
plt.ylim(-4, 0.5)

###############################################################################
plt.subplot(2, 2, 3)
plt.plot(df['y+'], targets, 'k.',  label='DNS')
plt.title('Predicted Mean with 90% Uncertainty Bound', fontsize=12)
# predictive mean and deviation
plt.plot(df['y+'], MLP_mean, 'k-', linewidth=1.5, label='MLP Prediction')
plt.plot(df['y+'], MLP_Re_mean, 'g:', linewidth=1.5, label='MLP-Re Prediction')
plt.plot(df['y+'], MLP_BC_Re_mean, 'c:', linewidth=1.5, label='MLP-BC-Re Prediction')
#plt.plot(df['y+'], MLP_BC_Re_NL_mean, 'm:', linewidth=1.5, label='MLP-BC-Re-NL Prediction')
plt.plot(df['y+'], BootMLP_Prior1_mean, 'b--', linewidth=1.5, label='MLP-BC-Re-Prior1 Prediction')
plt.fill_between(df['y+'], BootMLP_Prior1_mean - BootMLP_Prior1_std, BootMLP_Prior1_mean + BootMLP_Prior1_std, alpha=0.5, color='red')
plt.fill_between(df['y+'], BootMLP_Prior1_mean + 1.64*BootMLP_Prior1_std, BootMLP_Prior1_mean - 1.64*BootMLP_Prior1_std, alpha=0.15, color='red') # 90% uncertainty: 1.64 stdinty: 1.64 std
plt.legend(loc='lower left')
plt.xscale("log")
plt.xlabel('y+'); plt.ylabel('log($a_{uv}$)')
plt.xlim(-100,6000)
plt.ylim(-4, 0.5)
###############################################################################
plt.subplot(2, 2, 4)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Prediction of Each Ensemble', fontsize=12)
plt.xlabel('y+'); plt.ylabel('$log(a_{uv})$')
plt.legend(loc='lower left')

# loop for predictions of each head 
for i in range(ensemble):
    plt.plot(df['y+'], BootMLP_Prior1_preds[i,:], linestyle='--', linewidth=1.5)
plt.xscale("log")
plt.xlim(-100, 800)
plt.ylim(-4, 0.5);


In [ ]:
print('Loss for MLP')
fig = plt.figure(dpi=300)
plt.subplot(2, 1, 1)
plot_loss_history(MLP_loss[0], loglog=True)
plt.xlim(-20, 1000)

plt.subplot(2, 1, 2)
plot_loss_history(MLP_loss[0], loglog=True)
plt.xlim(-20, 300)


print('Loss for MLP-Re')
fig = plt.figure(dpi=300)
plt.subplot(2, 1, 1)
plot_loss_history(MLP_Re_loss[0], loglog=True)
plt.xlim(-20, 1000)

plt.subplot(2, 1, 2)
plot_loss_history(MLP_Re_loss[0], loglog=True)
plt.xlim(-20, 100)


print('Loss for MLP-BC-Re')
fig = plt.figure(dpi=300)
plt.subplot(2, 1, 1)
plot_loss_history(MLP_BC_Re_loss[0], loglog=True)
plt.xlim(-20, 1000)

plt.subplot(2, 1, 2)
plot_loss_history(MLP_BC_Re_loss[0], loglog=True)
plt.xlim(-20, 100)


#print('Loss for MLP-BC-Re-NL')
#fig = plt.figure(dpi=300)
#plt.subplot(2, 1, 1)
#for i in range(ensemble):
#    plot_loss_history(MLP_BC_Re_NL_loss[i], loglog=False)
#plt.xlim(-20, 1000)

#plt.subplot(2, 1, 2)
#for i in range(ensemble):
#    plot_loss_history(MLP_BC_Re_NL_loss[i], loglog=False)
#plt.xlim(-20, 100)


print('Loss for Bootstrap for MLP-BC-Re-Prior1')
fig = plt.figure(dpi=300)
plt.subplot(2, 1, 1)
for i in range(ensemble):
    plot_loss_history(BootMLP_Prior1_loss[i], loglog=True)
plt.xlim(-20, 1000)

plt.subplot(2, 1, 2)
for i in range(ensemble):
    plot_loss_history(BootMLP_Prior1_loss[i], loglog=True)
plt.xlim(-20, 100)

print('Loss for Bootstrap for MLP-BC-Re-Prior2')
fig = plt.figure(dpi=300)
plt.subplot(2, 1, 1)
for i in range(ensemble):
    plot_loss_history(BootMLP_Prior2_loss[i], loglog=True)
plt.xlim(-20, 1000)

plt.subplot(2, 1, 2)
for i in range(ensemble):
    plot_loss_history(BootMLP_Prior2_loss[i], loglog=True)
plt.xlim(-20, 100)

print('Loss for Bootstrap for MLP-BC-Re-Prior3')
fig = plt.figure(dpi=300)
plt.subplot(2, 1, 1)
for i in range(ensemble):
    plot_loss_history(BootMLP_Prior3_loss[i], loglog=True)
plt.xlim(-20, 1000)

plt.subplot(2, 1, 2)
for i in range(ensemble):
    plot_loss_history(BootMLP_Prior3_loss[i], loglog=True)
plt.xlim(-20, 100)

print('Loss for Bootstrap for MLP-BC-Re-Prior5')
fig = plt.figure(dpi=300)
plt.subplot(2, 1, 1)
for i in range(ensemble):
    plot_loss_history(BootMLP_Prior5_loss[i], loglog=True)
plt.xlim(-20, 1000)

plt.subplot(2, 1, 2)
for i in range(ensemble):
    plot_loss_history(BootMLP_Prior5_loss[i], loglog=True)
plt.xlim(-20, 100)

print('Loss for Bootstrap for MLP-BC-Re-Prior7')
fig = plt.figure(dpi=300)
plt.subplot(2, 1, 1)
for i in range(ensemble):
    plot_loss_history(BootMLP_Prior7_loss[i], loglog=True)
plt.xlim(-20, 1000)

plt.subplot(2, 1, 2)
for i in range(ensemble):
    plot_loss_history(BootMLP_Prior7_loss[i], loglog=True)
plt.xlim(-20, 100)

print('Loss for Bootstrap for MLP-BC-Re-Prior10')
fig = plt.figure(dpi=300)
plt.subplot(2, 1, 1)
for i in range(ensemble):
    plot_loss_history(BootMLP_Prior10_loss[i], loglog=True)
plt.xlim(-20, 1000)

plt.subplot(2, 1, 2)
for i in range(ensemble):
    plot_loss_history(BootMLP_Prior10_loss[i], loglog=True)
plt.xlim(-20, 100)

In [ ]:
# save model
MLP.save_checkpoint('MLPcheckpoint.pt')
MLP_Re.save_checkpoint('MLP_Re_checkpoint.pt')
MLP_BC_Re.save_checkpoint('MLP_BC_Re_checkpoint.pt')
BootMLP_Prior1.save_checkpoint('MLP_BC_Re_Prior1_checkpoint.pt')

In [ ]:
print('Prior=2')
fig = plt.figure(figsize=[12,10], dpi=300)
fig.suptitle('Uncertainty Estimates by Bootstrapped Ensemble', verticalalignment='center')
###############################################################################
plt.subplot(2, 2, 1)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Predicted Mean with 90% Uncertainty Bound', fontsize=12)
plt.xlim(-100, 800)
plt.ylim(-4, 0.5)
# predictive mean and deviation
plt.plot(df['y+'], MLP_mean, 'k-', linewidth=1.5, label='MLP Prediction')
plt.plot(df['y+'], MLP_Re_mean, 'g:', linewidth=1.5, label='MLP-Re Prediction')
plt.plot(df['y+'], MLP_BC_Re_mean, 'c:', linewidth=1.5, label='MLP-BC-Re Prediction')
#plt.plot(df['y+'], MLP_BC_Re_NL_mean, 'm:', linewidth=1.5, label='MLP-BC-Re-NL Prediction')
plt.plot(df['y+'], BootMLP_Prior2_mean, 'b--', linewidth=1.5, label='MLP-BC-Re Prior=2 Prediction')
plt.fill_between(df['y+'], BootMLP_Prior2_mean - BootMLP_Prior2_std, BootMLP_Prior2_mean + BootMLP_Prior2_std, alpha=0.5, color='red')
plt.fill_between(df['y+'], BootMLP_Prior2_mean + 1.64*BootMLP_Prior2_std, BootMLP_Prior2_mean - 1.64*BootMLP_Prior2_std, alpha=0.15, color='red') # 90% uncertainty: 1.64 std
plt.legend(loc='lower left')
plt.xlabel('y+'); plt.ylabel('$a_{uv}$')
###############################################################################
plt.subplot(2, 2, 2)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Prediction of Each Ensemble', fontsize=12)
plt.xlabel('y+'); plt.ylabel('$a_{uv}$')
plt.legend(loc='lower left')

# loop for predictions of each head 
for i in range(ensemble):
    plt.plot(df['y+'], BootMLP_Prior2_preds[i,:], linestyle='--', linewidth=1.5)
plt.xlim(-100, 800)
plt.ylim(-4, 0.5)

###############################################################################
plt.subplot(2, 2, 3)
plt.plot(df['y+'], targets, 'k.',  label='DNS')
plt.title('Predicted Mean with 90% Uncertainty Bound', fontsize=12)
# predictive mean and deviation
plt.plot(df['y+'], MLP_mean, 'k-', linewidth=1.5, label='MLP Prediction')
plt.plot(df['y+'], MLP_Re_mean, 'g:', linewidth=1.5, label='MLP-Re Prediction')
plt.plot(df['y+'], MLP_BC_Re_mean, 'c:', linewidth=1.5, label='MLP-BC-Re Prediction')
#plt.plot(df['y+'], MLP_BC_Re_NL_mean, 'm:', linewidth=1.5, label='MLP-BC-Re-NL Prediction')
plt.plot(df['y+'], BootMLP_Prior2_mean, 'b--', linewidth=1.5, label='MLP-BC-Re Prior=2 Prediction')
plt.fill_between(df['y+'], BootMLP_Prior2_mean - BootMLP_Prior2_std, BootMLP_Prior2_mean + BootMLP_Prior2_std, alpha=0.5, color='red')
plt.fill_between(df['y+'], BootMLP_Prior2_mean + 1.64*BootMLP_Prior2_std, BootMLP_Prior2_mean - 1.64*BootMLP_Prior2_std, alpha=0.15, color='red') # 90% uncertainty: 1.64 stdinty: 1.64 std
plt.legend(loc='lower left')
plt.xscale("log")
plt.xlabel('y+'); plt.ylabel('log($a_{uv}$)')
plt.xlim(-100,6000)
plt.ylim(-4, 0.5)
###############################################################################
plt.subplot(2, 2, 4)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Prediction of Each Ensemble', fontsize=12)
plt.xlabel('y+'); plt.ylabel('$log(a_{uv})$')
plt.legend(loc='lower left')

# loop for predictions of each head 
for i in range(ensemble):
    plt.plot(df['y+'], BootMLP_Prior2_preds[i,:], linestyle='--', linewidth=1.5)
plt.xscale("log")
plt.xlim(-100, 800)
plt.ylim(-4, 0.5);

In [ ]:
print('Prior=3')
fig = plt.figure(figsize=[12,10], dpi=300)
fig.suptitle('Uncertainty Estimates by Bootstrapped Ensemble', verticalalignment='center')
###############################################################################
plt.subplot(2, 2, 1)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Predicted Mean with 90% Uncertainty Bound', fontsize=12)
plt.xlim(-100, 800)
plt.ylim(-4, 0.5)
# predictive mean and deviation
plt.plot(df['y+'], MLP_mean, 'k-', linewidth=1.5, label='MLP Prediction')
plt.plot(df['y+'], MLP_Re_mean, 'g:', linewidth=1.5, label='MLP-Re Prediction')
plt.plot(df['y+'], MLP_BC_Re_mean, 'c:', linewidth=1.5, label='MLP-BC-Re Prediction')
#plt.plot(df['y+'], MLP_BC_Re_NL_mean, 'm:', linewidth=1.5, label='MLP-BC-Re-NL Prediction')
plt.plot(df['y+'], BootMLP_Prior3_mean, 'b--', linewidth=1.5, label='MLP-BC-Re Prior=3 Prediction')
plt.fill_between(df['y+'], BootMLP_Prior3_mean - BootMLP_Prior3_std, BootMLP_Prior3_mean + BootMLP_Prior3_std, alpha=0.5, color='red')
plt.fill_between(df['y+'], BootMLP_Prior3_mean + 1.64*BootMLP_Prior3_std, BootMLP_Prior3_mean - 1.64*BootMLP_Prior3_std, alpha=0.15, color='red') # 90% uncertainty: 1.64 std
plt.legend(loc='lower left')
plt.xlabel('y+'); plt.ylabel('$a_{uv}$')
###############################################################################
plt.subplot(2, 2, 2)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Prediction of Each Ensemble', fontsize=12)
plt.xlabel('y+'); plt.ylabel('$a_{uv}$')
plt.legend(loc='lower left')

# loop for predictions of each head 
for i in range(ensemble):
    plt.plot(df['y+'], BootMLP_Prior3_preds[i,:], linestyle='--', linewidth=1.5)
plt.xlim(-100, 800)
plt.ylim(-4, 0.5)

###############################################################################
plt.subplot(2, 2, 3)
plt.plot(df['y+'], targets, 'k.',  label='DNS')
plt.title('Predicted Mean with 90% Uncertainty Bound', fontsize=12)
# predictive mean and deviation
plt.plot(df['y+'], MLP_mean, 'k-', linewidth=1.5, label='MLP Prediction')
plt.plot(df['y+'], MLP_Re_mean, 'g:', linewidth=1.5, label='MLP-Re Prediction')
plt.plot(df['y+'], MLP_BC_Re_mean, 'c:', linewidth=1.5, label='MLP-BC-Re Prediction')
#plt.plot(df['y+'], MLP_BC_Re_NL_mean, 'm:', linewidth=1.5, label='MLP-BC-Re-NL Prediction')
plt.plot(df['y+'], BootMLP_Prior3_mean, 'b--', linewidth=1.5, label='MLP-BC-Re Prior=3 Prediction')
plt.fill_between(df['y+'], BootMLP_Prior3_mean - BootMLP_Prior3_std, BootMLP_Prior3_mean + BootMLP_Prior3_std, alpha=0.5, color='red')
plt.fill_between(df['y+'], BootMLP_Prior3_mean + 1.64*BootMLP_Prior3_std, BootMLP_Prior3_mean - 1.64*BootMLP_Prior3_std, alpha=0.15, color='red') # 90% uncertainty: 1.64 stdinty: 1.64 std
plt.legend(loc='lower left')
plt.xscale("log")
plt.xlabel('y+'); plt.ylabel('log($a_{uv}$)')
plt.xlim(-100,6000)
plt.ylim(-4, 0.5)
###############################################################################
plt.subplot(2, 2, 4)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Prediction of Each Ensemble', fontsize=12)
plt.xlabel('y+'); plt.ylabel('$log(a_{uv})$')
plt.legend(loc='lower left')

# loop for predictions of each head 
for i in range(ensemble):
    plt.plot(df['y+'], BootMLP_Prior3_preds[i,:], linestyle='--', linewidth=1.5)
plt.xscale("log")
plt.xlim(-100, 800)
plt.ylim(-4, 0.5);

In [ ]:
print('Prior=5')
fig = plt.figure(figsize=[12,10], dpi=300)
fig.suptitle('Uncertainty Estimates by Bootstrapped Ensemble', verticalalignment='center')
###############################################################################
plt.subplot(2, 2, 1)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Predicted Mean with 90% Uncertainty Bound', fontsize=12)
plt.xlim(-100, 800)
plt.ylim(-4, 0.5)
# predictive mean and deviation
plt.plot(df['y+'], MLP_mean, 'k-', linewidth=1.5, label='MLP Prediction')
plt.plot(df['y+'], MLP_Re_mean, 'g:', linewidth=1.5, label='MLP-Re Prediction')
plt.plot(df['y+'], MLP_BC_Re_mean, 'c:', linewidth=1.5, label='MLP-BC-Re Prediction')
#plt.plot(df['y+'], MLP_BC_Re_NL_mean, 'm:', linewidth=1.5, label='MLP-BC-Re-NL Prediction')
plt.plot(df['y+'], BootMLP_Prior2_mean, 'b--', linewidth=1.5, label='MLP-BC-Re Prior=5 Prediction')
plt.fill_between(df['y+'], BootMLP_Prior5_mean - BootMLP_Prior5_std, BootMLP_Prior5_mean + BootMLP_Prior5_std, alpha=0.5, color='red')
plt.fill_between(df['y+'], BootMLP_Prior5_mean + 1.64*BootMLP_Prior5_std, BootMLP_Prior5_mean - 1.64*BootMLP_Prior5_std, alpha=0.15, color='red') # 90% uncertainty: 1.64 std
plt.legend(loc='lower left')
plt.xlabel('y+'); plt.ylabel('$a_{uv}$')
###############################################################################
plt.subplot(2, 2, 2)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Prediction of Each Ensemble', fontsize=12)
plt.xlabel('y+'); plt.ylabel('$a_{uv}$')
plt.legend(loc='lower left')

# loop for predictions of each head 
for i in range(ensemble):
    plt.plot(df['y+'], BootMLP_Prior5_preds[i,:], linestyle='--', linewidth=1.5)
plt.xlim(-100, 800)
plt.ylim(-4, 0.5)

###############################################################################
plt.subplot(2, 2, 3)
plt.plot(df['y+'], targets, 'k.',  label='DNS')
plt.title('Predicted Mean with 90% Uncertainty Bound', fontsize=12)
# predictive mean and deviation
plt.plot(df['y+'], MLP_mean, 'k-', linewidth=1.5, label='MLP Prediction')
plt.plot(df['y+'], MLP_Re_mean, 'g:', linewidth=1.5, label='MLP-Re Prediction')
plt.plot(df['y+'], MLP_BC_Re_mean, 'c:', linewidth=1.5, label='MLP-BC-Re Prediction')
#plt.plot(df['y+'], MLP_BC_Re_NL_mean, 'm:', linewidth=1.5, label='MLP-BC-Re-NL Prediction')
plt.plot(df['y+'], BootMLP_Prior5_mean, 'b--', linewidth=1.5, label='MLP-BC-Re Prior=5 Prediction')
plt.fill_between(df['y+'], BootMLP_Prior5_mean - BootMLP_Prior5_std, BootMLP_Prior5_mean + BootMLP_Prior5_std, alpha=0.5, color='red')
plt.fill_between(df['y+'], BootMLP_Prior5_mean + 1.64*BootMLP_Prior5_std, BootMLP_Prior5_mean - 1.64*BootMLP_Prior5_std, alpha=0.15, color='red') # 90% uncertainty: 1.64 stdinty: 1.64 std
plt.legend(loc='lower left')
plt.xscale("log")
plt.xlabel('y+'); plt.ylabel('log($a_{uv}$)')
plt.xlim(-100,6000)
plt.ylim(-4, 0.5)
###############################################################################
plt.subplot(2, 2, 4)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Prediction of Each Ensemble', fontsize=12)
plt.xlabel('y+'); plt.ylabel('$log(a_{uv})$')
plt.legend(loc='lower left')

# loop for predictions of each head 
for i in range(ensemble):
    plt.plot(df['y+'], BootMLP_Prior5_preds[i,:], linestyle='--', linewidth=1.5)
plt.xscale("log")
plt.xlim(-100, 800)
plt.ylim(-4, 0.5);

In [ ]:
print('Prior=7')
fig = plt.figure(figsize=[12,10], dpi=300)
fig.suptitle('Uncertainty Estimates by Bootstrapped Ensemble', verticalalignment='center')
###############################################################################
plt.subplot(2, 2, 1)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Predicted Mean with 90% Uncertainty Bound', fontsize=12)
plt.xlim(-100, 800)
plt.ylim(-4, 0.5)
# predictive mean and deviation
plt.plot(df['y+'], MLP_mean, 'k-', linewidth=1.5, label='MLP Prediction')
plt.plot(df['y+'], MLP_Re_mean, 'g:', linewidth=1.5, label='MLP-Re Prediction')
plt.plot(df['y+'], MLP_BC_Re_mean, 'c:', linewidth=1.5, label='MLP-BC-Re Prediction')
#plt.plot(df['y+'], MLP_BC_Re_NL_mean, 'm:', linewidth=1.5, label='MLP-BC-Re-NL Prediction')
plt.plot(df['y+'], BootMLP_Prior7_mean, 'b--', linewidth=1.5, label='MLP-BC-Re Prior=7 Prediction')
plt.fill_between(df['y+'], BootMLP_Prior7_mean - BootMLP_Prior7_std, BootMLP_Prior7_mean + BootMLP_Prior7_std, alpha=0.5, color='red')
plt.fill_between(df['y+'], BootMLP_Prior7_mean + 1.64*BootMLP_Prior7_std, BootMLP_Prior7_mean - 1.64*BootMLP_Prior7_std, alpha=0.15, color='red') # 90% uncertainty: 1.64 std
plt.legend(loc='lower left')
plt.xlabel('y+'); plt.ylabel('$a_{uv}$')
###############################################################################
plt.subplot(2, 2, 2)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Prediction of Each Ensemble', fontsize=12)
plt.xlabel('y+'); plt.ylabel('$a_{uv}$')
plt.legend(loc='lower left')

# loop for predictions of each head 
for i in range(ensemble):
    plt.plot(df['y+'], BootMLP_Prior7_preds[i,:], linestyle='--', linewidth=1.5)
plt.xlim(-100, 800)
plt.ylim(-4, 0.5)

###############################################################################
plt.subplot(2, 2, 3)
plt.plot(df['y+'], targets, 'k.',  label='DNS')
plt.title('Predicted Mean with 90% Uncertainty Bound', fontsize=12)
# predictive mean and deviation
plt.plot(df['y+'], MLP_mean, 'k-', linewidth=1.5, label='MLP Prediction')
plt.plot(df['y+'], MLP_Re_mean, 'g:', linewidth=1.5, label='MLP-Re Prediction')
plt.plot(df['y+'], MLP_BC_Re_mean, 'c:', linewidth=1.5, label='MLP-BC-Re Prediction')
#plt.plot(df['y+'], MLP_BC_Re_NL_mean, 'm:', linewidth=1.5, label='MLP-BC-Re-NL Prediction')
plt.plot(df['y+'], BootMLP_Prior7_mean, 'b--', linewidth=1.5, label='MLP-BC-Re Prior=7 Prediction')
plt.fill_between(df['y+'], BootMLP_Prior7_mean - BootMLP_Prior7_std, BootMLP_Prior7_mean + BootMLP_Prior7_std, alpha=0.5, color='red')
plt.fill_between(df['y+'], BootMLP_Prior7_mean + 1.64*BootMLP_Prior7_std, BootMLP_Prior7_mean - 1.64*BootMLP_Prior7_std, alpha=0.15, color='red') # 90% uncertainty: 1.64 stdinty: 1.64 std
plt.legend(loc='lower left')
plt.xscale("log")
plt.xlabel('y+'); plt.ylabel('log($a_{uv}$)')
plt.xlim(-100,6000)
plt.ylim(-4, 0.5)
###############################################################################
plt.subplot(2, 2, 4)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Prediction of Each Ensemble', fontsize=12)
plt.xlabel('y+'); plt.ylabel('$log(a_{uv})$')
plt.legend(loc='lower left')

# loop for predictions of each head 
for i in range(ensemble):
    plt.plot(df['y+'], BootMLP_Prior7_preds[i,:], linestyle='--', linewidth=1.5)
plt.xscale("log")
plt.xlim(-100, 800)
plt.ylim(-4, 0.5);

In [ ]:
print('Prior=10')
fig = plt.figure(figsize=[12,10], dpi=300)
fig.suptitle('Uncertainty Estimates by Bootstrapped Ensemble', verticalalignment='center')
###############################################################################
plt.subplot(2, 2, 1)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Predicted Mean with 90% Uncertainty Bound', fontsize=12)
plt.xlim(-100, 800)
plt.ylim(-4, 0.5)
# predictive mean and deviation
plt.plot(df['y+'], MLP_mean, 'k-', linewidth=1.5, label='MLP Prediction')
plt.plot(df['y+'], MLP_Re_mean, 'g:', linewidth=1.5, label='MLP-Re Prediction')
plt.plot(df['y+'], MLP_BC_Re_mean, 'c:', linewidth=1.5, label='MLP-BC-Re Prediction')
#plt.plot(df['y+'], MLP_BC_Re_NL_mean, 'm:', linewidth=1.5, label='MLP-BC-Re-NL Prediction')
plt.plot(df['y+'], BootMLP_Prior10_mean, 'b--', linewidth=1.5, label='MLP-BC-Re Prior=10 Prediction')
plt.fill_between(df['y+'], BootMLP_Prior10_mean - BootMLP_Prior10_std, BootMLP_Prior10_mean + BootMLP_Prior10_std, alpha=0.5, color='red')
plt.fill_between(df['y+'], BootMLP_Prior10_mean + 1.64*BootMLP_Prior10_std, BootMLP_Prior10_mean - 1.64*BootMLP_Prior10_std, alpha=0.15, color='red') # 90% uncertainty: 1.64 std
plt.legend(loc='lower left')
plt.xlabel('y+'); plt.ylabel('$a_{uv}$')
###############################################################################
plt.subplot(2, 2, 2)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Prediction of Each Ensemble', fontsize=12)
plt.xlabel('y+'); plt.ylabel('$a_{uv}$')
plt.legend(loc='lower left')

# loop for predictions of each head 
for i in range(ensemble):
    plt.plot(df['y+'], BootMLP_Prior10_preds[i,:], linestyle='--', linewidth=1.5)
plt.xlim(-100, 800)
plt.ylim(-4, 0.5)

###############################################################################
plt.subplot(2, 2, 3)
plt.plot(df['y+'], targets, 'k.',  label='DNS')
plt.title('Predicted Mean with 90% Uncertainty Bound', fontsize=12)
# predictive mean and deviation
plt.plot(df['y+'], MLP_mean, 'k-', linewidth=1.5, label='MLP Prediction')
plt.plot(df['y+'], MLP_Re_mean, 'g:', linewidth=1.5, label='MLP-Re Prediction')
plt.plot(df['y+'], MLP_BC_Re_mean, 'c:', linewidth=1.5, label='MLP-BC-Re Prediction')
#plt.plot(df['y+'], MLP_BC_Re_NL_mean, 'm:', linewidth=1.5, label='MLP-BC-Re-NL Prediction')
plt.plot(df['y+'], BootMLP_Prior10_mean, 'b--', linewidth=1.5, label='MLP-BC-Re Prior=10 Prediction')
plt.fill_between(df['y+'], BootMLP_Prior10_mean - BootMLP_Prior10_std, BootMLP_Prior10_mean + BootMLP_Prior10_std, alpha=0.5, color='red')
plt.fill_between(df['y+'], BootMLP_Prior10_mean + 1.64*BootMLP_Prior10_std, BootMLP_Prior10_mean - 1.64*BootMLP_Prior10_std, alpha=0.15, color='red') # 90% uncertainty: 1.64 stdinty: 1.64 std
plt.legend(loc='lower left')
plt.xscale("log")
plt.xlabel('y+'); plt.ylabel('log($a_{uv}$)')
plt.xlim(-100,6000)
plt.ylim(-4, 0.5)
###############################################################################
plt.subplot(2, 2, 4)
plt.plot(df['y+'], targets, 'k.', label='DNS')
plt.title('Prediction of Each Ensemble', fontsize=12)
plt.xlabel('y+'); plt.ylabel('$log(a_{uv})$')
plt.legend(loc='lower left')

# loop for predictions of each head 
for i in range(ensemble):
    plt.plot(df['y+'], BootMLP_Prior10_preds[i,:], linestyle='--', linewidth=1.5)
plt.xscale("log")
plt.xlim(-100, 800)
plt.ylim(-4, 0.5);